In [13]:
import os
import zipfile
import requests
import shutil
import json

# --- İndirme Fonksiyonu ---
def download_and_extract(url, download_path, extract_to):
    os.makedirs(download_path, exist_ok=True)
    filename = url.split('/')[-1]
    file_path = os.path.join(download_path, filename)
    
    # Dosya indirme
    if not os.path.exists(file_path):
        print(f"İndiriliyor: {filename}")
        response = requests.get(url, stream=True)
        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"İndirildi: {filename}")
    
    # Dosya çıkarma
    print(f"Çıkartılıyor: {filename}")
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Çıkartıldı: {filename}")

# --- Ana İşlem ---
def create_mini_coco_subset():
    download_dir = 'coco'
    extract_dir = 'coco'
    
    urls = {
        "val2017.zip": "http://images.cocodataset.org/zips/val2017.zip",
        "annotations_trainval2017.zip": "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
    }
    
    for url in urls.values():
        download_and_extract(url, download_dir, extract_dir)
    
    # İlk 1000 resmi seçelim
    val_images_dir = os.path.join(extract_dir, 'val2017')
    images = sorted(os.listdir(val_images_dir))[:1000]
    
    mini_images_dir = os.path.join(extract_dir, 'mini_val2017')
    os.makedirs(mini_images_dir, exist_ok=True)
    
    print("İlk 1000 resim kopyalanıyor...")
    for img in images:
        src = os.path.join(val_images_dir, img)
        dst = os.path.join(mini_images_dir, img)
        shutil.copy(src, dst)
    print("Resim kopyalama tamamlandı.")

    # Annotation dosyasını okuma
    with open(os.path.join(extract_dir, 'annotations', 'instances_val2017.json')) as f:
        coco_data = json.load(f)

    # İlk 1000 resme ait yeni annotation dosyası oluşturma
    image_id_set = set()
    mini_images = []
    for img in coco_data['images']:
        if img['file_name'] in images:
            mini_images.append(img)
            image_id_set.add(img['id'])

    mini_annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] in image_id_set]

    mini_coco = {
        'info': coco_data['info'],
        'licenses': coco_data['licenses'],
        'categories': coco_data['categories'],
        'images': mini_images,
        'annotations': mini_annotations,
    }

    mini_annotation_path = os.path.join(extract_dir, 'annotations', 'mini_instances_val2017.json')
    with open(mini_annotation_path, 'w') as f:
        json.dump(mini_coco, f)
    
    print(f"Yeni mini COCO annotation dosyası kaydedildi: {mini_annotation_path}")
    print("✅ Mini COCO subset hazır!")

# --- Çalıştır ---
if __name__ == "__main__":
    create_mini_coco_subset()

İndiriliyor: val2017.zip
İndirildi: val2017.zip
Çıkartılıyor: val2017.zip
Çıkartıldı: val2017.zip
İndiriliyor: annotations_trainval2017.zip
İndirildi: annotations_trainval2017.zip
Çıkartılıyor: annotations_trainval2017.zip
Çıkartıldı: annotations_trainval2017.zip
İlk 1000 resim kopyalanıyor...
Resim kopyalama tamamlandı.
Yeni mini COCO annotation dosyası kaydedildi: coco/annotations/mini_instances_val2017.json
✅ Mini COCO subset hazır!


In [17]:
import os
import json

# --- Yollar ---
full_annotations_path = 'coco/annotations/instances_val2017.json'
subset_images_folder = 'coco/mini_val2017'
save_subset_json_path = 'coco/annotations/mini_instances_val2017.json'

# --- Resim isimlerini oku ---
subset_images = sorted(os.listdir(subset_images_folder))
subset_image_names = set(subset_images)

# --- Annotation dosyasını yükle ---
with open(full_annotations_path, 'r') as f:
    full_coco = json.load(f)

# --- Yeni JSON içeriklerini oluştur ---
subset_images_info = []
subset_annotations_info = []
valid_image_ids = set()

# Sadece subsette olan resimler
for img_info in full_coco['images']:
    if img_info['file_name'] in subset_image_names:
        subset_images_info.append(img_info)
        valid_image_ids.add(img_info['id'])

# Bu resimlere ait annotation'ları seç
for ann_info in full_coco['annotations']:
    if ann_info['image_id'] in valid_image_ids:
        subset_annotations_info.append(ann_info)

# --- Alt küme JSON'u oluştur ---
mini_coco = {
    'info': full_coco['info'],
    'licenses': full_coco['licenses'],
    'categories': full_coco['categories'],
    'images': subset_images_info,
    'annotations': subset_annotations_info
}

# --- JSON dosyasını kaydet ---
os.makedirs(os.path.dirname(save_subset_json_path), exist_ok=True)

with open(save_subset_json_path, 'w') as f:
    json.dump(mini_coco, f)

print("✅ Mini annotation dosyası oluşturuldu:", save_subset_json_path)

✅ Mini annotation dosyası oluşturuldu: coco/annotations/mini_instances_val2017.json


In [19]:
import os
import json

# --- Dosya yolları ---
mini_images_folder = 'coco/mini_val2017'
mini_annotations_path = 'coco/annotations/mini_instances_val2017.json'

# --- Yükle ---
with open(mini_annotations_path, 'r') as f:
    mini_coco = json.load(f)

# --- Klasördeki resimleri oku ---
image_filenames = sorted(os.listdir(mini_images_folder))
image_filenames_set = set(image_filenames)

# --- JSON'daki resimleri oku ---
json_filenames_set = set(img['file_name'] for img in mini_coco['images'])
json_image_ids_set = set(img['id'] for img in mini_coco['images'])

# --- 1. Her dosya JSON'da var mı? ---
missing_in_json = image_filenames_set - json_filenames_set
if missing_in_json:
    print(f"❌ Bu klasörde olan ama JSON'da olmayan dosyalar var: {missing_in_json}")
else:
    print("✅ Tüm resimler JSON'da kayıtlı.")

# --- 2. Her JSON resmi klasörde var mı? ---
missing_in_folder = json_filenames_set - image_filenames_set
if missing_in_folder:
    print(f"❌ JSON'da olup klasörde bulunmayan resimler var: {missing_in_folder}")
else:
    print("✅ Tüm JSON resimleri klasörde mevcut.")

# --- 3. Annotationlar doğru resimlere bağlı mı? ---
invalid_annotations = []
for ann in mini_coco['annotations']:
    if ann['image_id'] not in json_image_ids_set:
        invalid_annotations.append(ann['id'])

if invalid_annotations:
    print(f"❌ Bazı annotationlar geçersiz image_id'lere bağlı: {invalid_annotations}")
else:
    print("✅ Tüm annotationlar geçerli image_id'lere bağlı.")

print("🎯 Uyum kontrolü tamamlandı!")

✅ Tüm resimler JSON'da kayıtlı.
✅ Tüm JSON resimleri klasörde mevcut.
✅ Tüm annotationlar geçerli image_id'lere bağlı.
🎯 Uyum kontrolü tamamlandı!
